get nconst from imdbnames_df link with imdbprincipals on nconst get tconst to get movie and possibly call imdb_api calculate cr and wicar for each actor.

In [1]:
import pandas as pd
import imdb_api as ia

In [2]:
imdbtitle_df = pd.read_csv('./data/test/title.basics.tsv', sep='\t',encoding='utf-8',dtype='str')

In [3]:
imdbnames_df = pd.read_csv('./data/test/name.basics.tsv', sep='\t',encoding='utf-8')

In [4]:
wicar_df = pd.read_csv('./data/actorsinfluence_wicar.csv', encoding='utf-8', index_col=0)
wicar_df.dropna(inplace=True)
wicar_df.sort_values(by='actor',ascending=True,inplace=True)
wicar_df.actor=wicar_df.actor.str.strip()
imdbtitle_df = imdbtitle_df.loc[imdbtitle_df.titleType == 'movie']

In [3]:
#imdbprincipals_df = pd.read_csv('./data/test/title.principals.tsv', sep='\t',encoding='utf-8')

In [12]:
del imdbprincipals_df

In [13]:
imdbprins_act_dirs_df = pd.read_pickle('./data/imdbprins_acts_dirs.pkl')

In [7]:
imdbprins_act_dirs_df = imdbprincipals_df[(imdbprincipals_df.category == 'actor') | 
                                          (imdbprincipals_df.category == 'actress') |
                                          (imdbprincipals_df.category == 'director') ]

In [9]:
imdbtitle_df = imdbtitle_df.loc[imdbtitle_df.titleType == 'movie']

In [ ]:
imdbnames_df.sort_values(by='primaryName',ascending=True,inplace=True)
imdbprins_act_dirs_df.sort_values(by=['nconst','tconst'],ascending=[True,True],inplace=True)
imdbtitle_df.sort_values(by=['tconst'],ascending=[True],inplace=True)

In [170]:
imdbnames_df.to_pickle('./data/imdbnames.pkl')

In [17]:
imdbtitle_df.to_pickle('./data/imdbtitle.pkl')

In [16]:
wicar_df.to_pickle('./data/wicar.pkl')

In [ ]:
imdbprins_act_dirs_df.to_pickle('./data/imdbprins_acts_dirs.pkl')

In [10]:
def get_actorid_byname(name):
    try:
        nid = imdbnames_df[imdbnames_df.primaryName == name].nconst.values[0]
        return nid
    except:
        return None

def get_movieids_from_nid(nameid):
    mids = imdbprins_act_dirs_df[imdbprins_act_dirs_df.nconst == nameid]['tconst'].values
    return mids


def get_movieinfo_from_mid(movieid):
    movieinfo = imdbtitle_df.loc[(imdbtitle_df.tconst == movieid)]
    return movieinfo


In [11]:
def get_all_movies():
    num=0
    movieinfos = pd.DataFrame()
    actors = wicar_df.actor.unique().tolist()
    nameid =''
    movieids=[]
    for actor in actors:
        #nameid or nconst from actor name lookup
        actid = get_actorid_byname(actor)
        #all movie_ids from the ncosnt
        movie_ids = get_movieids_from_nid(actid)


        if num%50 ==0:
            print('')
            print(num,'Found ',len(movie_ids),' movie ids for:',actor, actid)
        
        print('.', end = '')
        
        for mid in movie_ids:
            #for each movie id get the movie record  
            movieinfos = movieinfos.append(get_movieinfo_from_mid(mid))
        num+=1
    return movieinfos

In [ ]:
allmovies_df = get_all_movies()

In [15]:
%whos DataFrame

Variable                Type         Data/Info
----------------------------------------------
imdbnames_df            DataFrame                 nconst      <...>1144485 rows x 6 columns]
imdbprins_act_dirs_df   DataFrame                 tconst  orde<...>3702938 rows x 6 columns]
imdbtitle_df            DataFrame                tconst titleT<...>[584971 rows x 9 columns]
wicar_df                DataFrame             tconst          <...>\n[7189 rows x 8 columns]


In [ ]:
##Another approach.  join titles names and prins   
#check if name from wicar is in the joined table. to get the all the movies we need. to do the wicar influence

In [18]:
imdb_allmerged_df = imdbnames_df.merge(imdbprins_act_dirs_df.merge(imdbtitle_df,on='tconst'),on='nconst')

In [19]:
len(imdb_allmerged_df)

2373249

In [27]:
uniquenames = wicar_df.actor.unique().tolist()

In [28]:
filtered_allmerged_df = imdb_allmerged_df.loc[imdb_allmerged_df.primaryName.isin(uniquenames)]

In [29]:
len(filtered_allmerged_df)

56091

In [33]:
filtered_allmerged_df.sort_values(by=['primaryName', 'startYear'], ascending=[True,True], inplace=True)

<ipython-input-33-f10f6d980850>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_allmerged_df.sort_values(by=['primaryName', 'startYear'], ascending=[True,True], inplace=True)


In [34]:
filtered_allmerged_df

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles,tconst,ordering,category,job,characters,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
1453662,nm1265067,50 Cent,1975,\N,"soundtrack,actor,producer","tt3281796,tt3079380,tt1211956,tt0430308",tt0430308,1,actor,\N,"[""Marcus""]",movie,Get Rich or Die Tryin',Get Rich or Die Tryin',0,2005,\N,117,"Biography,Crime,Drama"
1453663,nm1265067,50 Cent,1975,\N,"soundtrack,actor,producer","tt3281796,tt3079380,tt1211956,tt0430308",tt0485920,2,actor,\N,"[""Black""]",movie,Vengeance,Jack's Law,0,2006,\N,90,"Action,Crime,Drama"
1453664,nm1265067,50 Cent,1975,\N,"soundtrack,actor,producer","tt3281796,tt3079380,tt1211956,tt0430308",tt0763840,2,actor,\N,"[""Jamal Aiken""]",movie,Home of the Brave,Home of the Brave,0,2006,\N,106,"Action,Drama,War"
1453665,nm1265067,50 Cent,1975,\N,"soundtrack,actor,producer","tt3281796,tt3079380,tt1211956,tt0430308",tt1034331,4,actor,\N,"[""Spider""]",movie,Righteous Kill,Righteous Kill,0,2008,\N,101,"Crime,Drama,Mystery"
1453668,nm1265067,50 Cent,1975,\N,"soundtrack,actor,producer","tt3281796,tt3079380,tt1211956,tt0430308",tt1311699,3,actor,\N,"[""Thigo""]",movie,Dead Man Running,Dead Man Running,0,2009,\N,92,"Action,Crime,Drama"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1285498,nm1049982,Óscar Jaenada,1975,\N,"actor,producer","tt1298650,tt3005242,tt0480255,tt4052882",tt1843318,1,actor,\N,"[""Ice Cold Milo""]",movie,The Last Party,The Last Party,0,\N,\N,\N,"Comedy,Thriller"
1285500,nm1049982,Óscar Jaenada,1975,\N,"actor,producer","tt1298650,tt3005242,tt0480255,tt4052882",tt2088716,1,actor,\N,"[""Pere""]",movie,Castells in the Air,Castells in the Air,0,\N,\N,98,Comedy
1285504,nm1049982,Óscar Jaenada,1975,\N,"actor,producer","tt1298650,tt3005242,tt0480255,tt4052882",tt3158294,1,actor,\N,"[""Michael Alvarez""]",movie,The Art of Living,The Art of Living,0,\N,\N,\N,Drama
1285509,nm1049982,Óscar Jaenada,1975,\N,"actor,producer","tt1298650,tt3005242,tt0480255,tt4052882",tt7826752,1,actor,\N,\N,movie,Turn,Turn,0,\N,\N,\N,Thriller


removed nconst as duplicate names. Harrison Ford, Katherine Stewart nm0829553, nm2123244, nm1960231,nm3819388, nm0243721,nm0615014,nm0599910,nm0019384,nm3086748,nm2695146

In [80]:
filtered_allmerged_df = filtered_allmerged_df.loc[filtered_allmerged_df.nconst != 'nm0932300']

In [96]:
filtered_allmerged_df.loc[filtered_allmerged_df.startYear <= '1950']

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles,tconst,ordering,category,job,characters,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
72209,nm0004463,Akkineni Nageshwara Rao,1923,2014,"actor,producer,miscellaneous","tt0260677,tt0253667,tt0259392,tt0251834",tt0155154,1,actor,\N,"[""Sri Rama""]",movie,Seeta Rama Jananam,Seeta Rama Jananam,0,1944,\N,\N,"Drama,Musical"
72295,nm0004463,Akkineni Nageshwara Rao,1923,2014,"actor,producer,miscellaneous","tt0260677,tt0253667,tt0259392,tt0251834",tt0259431,4,actor,\N,\N,movie,Mayalokam,Mayalokam,0,1945,\N,\N,"Drama,Musical,Romance"
72208,nm0004463,Akkineni Nageshwara Rao,1923,2014,"actor,producer,miscellaneous","tt0260677,tt0253667,tt0259392,tt0251834",tt0154910,4,actor,\N,"[""Prince Firoji, bhai of Somoji (Bezawada) & S...",movie,Mugguru Maratilu,Mugguru Maratilu,0,1946,\N,113,Drama
72206,nm0004463,Akkineni Nageshwara Rao,1923,2014,"actor,producer,miscellaneous","tt0260677,tt0253667,tt0259392,tt0251834",tt0154193,1,actor,\N,\N,movie,Balaraju,Balaraju,0,1948,\N,163,"Drama,Musical"
72267,nm0004463,Akkineni Nageshwara Rao,1923,2014,"actor,producer,miscellaneous","tt0260677,tt0253667,tt0259392,tt0251834",tt0255682,1,actor,\N,\N,movie,Vindhyarani,Vindhyarani,0,1948,\N,\N,"Drama,Musical"
72247,nm0004463,Akkineni Nageshwara Rao,1923,2014,"actor,producer,miscellaneous","tt0260677,tt0253667,tt0259392,tt0251834",tt0249104,1,actor,\N,"[""Prince Sudhakar""]",movie,Raksharekha,Raksharekha,0,1949,\N,168,"Drama,Romance"
72294,nm0004463,Akkineni Nageshwara Rao,1923,2014,"actor,producer,miscellaneous","tt0260677,tt0253667,tt0259392,tt0251834",tt0259392,1,actor,\N,"[""Qais""]",movie,Laila Majnu,Laila Majnu,0,1949,\N,171,"Drama,History,Musical"
72370,nm0004463,Akkineni Nageshwara Rao,1923,2014,"actor,producer,miscellaneous","tt0260677,tt0253667,tt0259392,tt0251834",tt0265261,2,actor,\N,\N,movie,Keelu Gurram,Keelu Gurram,0,1949,\N,220,"Drama,Fantasy"
72454,nm0004463,Akkineni Nageshwara Rao,1923,2014,"actor,producer,miscellaneous","tt0260677,tt0253667,tt0259392,tt0251834",tt9030016,4,actor,\N,\N,movie,Laila Majnu,Laila Majnu,0,1949,\N,169,Drama
72210,nm0004463,Akkineni Nageshwara Rao,1923,2014,"actor,producer,miscellaneous","tt0260677,tt0253667,tt0259392,tt0251834",tt0155173,1,actor,\N,\N,movie,Shri Lakshmamma Katha,Shri Lakshmamma Katha,0,1950,\N,\N,Drama


In [104]:
len(filtered_allmerged_df.tconst.unique().tolist())

36929

In [113]:
tconsts_80s = filtered_allmerged_df[(filtered_allmerged_df.startYear > '1979') & (filtered_allmerged_df.startYear < '1990')]['tconst'].tolist()

In [114]:
tconsts_90s = filtered_allmerged_df[(filtered_allmerged_df.startYear > '1989') & (filtered_allmerged_df.startYear < '2000')]['tconst'].tolist()

In [116]:
tconsts_10s = filtered_allmerged_df[(filtered_allmerged_df.startYear > '2009') & (filtered_allmerged_df.startYear < '2023')]['tconst'].tolist()

In [123]:
len(tconsts_80s)
len(set(tconsts_80s))

2908

In [124]:
len(set(tconsts_90s))

4849

In [125]:
len(set(tconsts_00s))

8033

In [126]:
len(set(tconsts_10s))

16299

In [127]:
filtered_allmerged_df.to_pickle('./data/filtered_allmerged.pkl')

######Use IMDB_API to query for all of the tconsts ids  Store in json files.

In [150]:
import imdb_api as ia
import time
moviesjsonfile_80s = './data/imdb_movies_80s.json'
moviesjsonfile_90s = './data/imdb_movies_90s.json'
moviesjsonfile_00s = './data/imdb_movies_00s.json'
moviesjsonfile_02s = './data/imdb_movies_02s.json'

moviesjsonfile_101s = './data/imdb_movies_101s.json'
moviesjsonfile_102s = './data/imdb_movies_102s.json'
moviesjsonfile_103s = './data/imdb_movies_103s.json'

In [157]:
#RUNS in 17 MINS for 3000 movies
#collect IMDB movie objects isung imdb_api
def collect_imdb_movies(movieids):
    counter = 0
    movies = []
    for id in movieids:
        try:
            movies.append(ia.search_by_id(id))
        except:
            print('Error Occurred')
            pass

        if counter%100 == 0:
            print(counter,id)
        counter +=1
    return movies

In [131]:
#list of MOVIE objects to json file
def movies_to_jsonfile(movies,file):
    movie_df = pd.DataFrame()
    for movie in movies:
        ## Use the MovieEncoder to decode a Movie object
        jsn = pd.read_json(StringIO(MovieEncoder().encode(movie)),
                           typ='series', encoding='utf-8')
        fr = jsn.to_frame()
        movie_df = movie_df.append(fr.T, ignore_index=True)
    movie_df.to_json(file)

In [161]:
moviesjsonfile_104s = './data/imdb_movies_104s.json'
moviesjsonfile_105s = './data/imdb_movies_105s.json'
moviesjsonfile_106s = './data/imdb_movies_106s.json'
moviesjsonfile_107s = './data/imdb_movies_107s.json'

In [140]:
import time
import json
from json import JSONEncoder
from imdb_api import Movie
from io import StringIO

# A specialised JSONEncoder that encodes Movie
# objects as JSON
class MovieEncoder(JSONEncoder):

    def default(self, object):

        if isinstance(object, Movie):
            return object.__dict__
        else:
            # call base class implementation which takes care of
            # raising exceptions for unsupported types
            return json.JSONEncoder.default(self, object)
        
movies_to_jsonfile(tmpmovies,moviesjsonfile_80s)

##########################

Lets import all the json files and create a data frame and merge with all filtereddf

In [168]:
def load_jsonfile_todf(jsonfile):
    with open(jsonfile) as f:
        #load json to dict to createDF
        json_df = pd.DataFrame(json.load(f))
    return json_df

In [170]:
queryied_80sto_20sdf = pd.DataFrame()

In [181]:
queryied_80sto_20sdf = queryied_80sto_20sdf.append(load_jsonfile_todf(moviesjsonfile_106s))

FileNotFoundError: [Errno 2] No such file or directory: './data/imdb_movies_106s.json'

In [184]:
queryied_80sto_20sdf.to_pickle('./data/queried80s20s.pkl')

In [ ]:
#Merge allfiltered with queried.   then make custom xls file to do actor/director influence 

In [189]:
big_mergeddf = filtered_allmerged_df.merge(queryied_80sto_20sdf,left_on='tconst', right_on='mid')

In [195]:
big_mergeddf.info(5)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 57238 entries, 0 to 57237
Data columns (total 44 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   nconst             57238 non-null  object 
 1   primaryName        57238 non-null  object 
 2   birthYear          57238 non-null  object 
 3   deathYear          57238 non-null  object 
 4   primaryProfession  57105 non-null  object 
 5   knownForTitles     57238 non-null  object 
 6   tconst             57238 non-null  object 
 7   ordering           57238 non-null  int64  
 8   category           57238 non-null  object 
 9   job                57238 non-null  object 
 10  characters         57238 non-null  object 
 11  titleType          57238 non-null  object 
 12  primaryTitle       57238 non-null  object 
 13  originalTitle      57238 non-null  object 
 14  isAdult            57238 non-null  object 
 15  startYear          57238 non-null  object 
 16  endYear            572

In [191]:
#then make custom xls file to do actor/director influence 
big_mergeddf.isAdult.value_counts()

0    56764
1      474
Name: isAdult, dtype: int64

In [200]:
big_mergeddf.sort_values(by=['primaryName', 'released'], ascending=[True,True], inplace=True)

In [202]:

bigactor_wicar_df = big_mergeddf[['tconst','title','released','nconst','primaryName','boxoffice']]


In [210]:
bigactor_wicar_df.to_excel('./data/bigactor_wicar.xls')

In [208]:
bigactor_wicar_df.drop_duplicates(inplace=True, ignore_index=True)

<ipython-input-208-ab8c726154d1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bigactor_wicar_df.drop_duplicates(inplace=True, ignore_index=True)


In [209]:
bigactor_wicar_df

,tconst,title,released,nconst,primaryName,boxoffice
0,tt0763840,Home of the Brave,05 Jan 2007,nm1265067,50 Cent,"$51,708"
1,tt1606390,All Things Fall Apart,05 Mar 2011,nm1265067,50 Cent,N/A
2,tt0485920,Vengeance,05 May 2016,nm1265067,50 Cent,N/A
3,tt1560954,Gun,05 Sep 2015,nm1265067,50 Cent,N/A
4,tt0430308,Get Rich or Die Tryin',09 Nov 2005,nm1265067,50 Cent,"$30,985,352"
...,...,...,...,...,...,...
42250,tt7689186,Let There Be Light,26 Sep 2019,nm4605787,Zuzana Konecná,N/A
42251,tt2121250,Alpha,08 Nov 2013,nm1974933,Álex Barahona,N/A
42252,tt1667108,Lo contrario al amor,26 Aug 2011,nm1974933,Álex Barahona,N/A
42253,tt2536448,Youth,03 Jul 2013,nm3260339,Émile Bertherat,N/A


In [213]:
wicaractor = pd.read_csv('./data/bigactor_wicar.csv',index_col=0)

In [218]:
wicaractor.loc[wicaractor.wicar == wicaractor.wicar.max()]

,tconst,title,released,nconst,primaryName,boxoffice,inf_adj_boxoffice,cr,wicar
40414,tt3704428,Untitled Elvis Presley Project,06-03-2022,nm0000158,Tom Hanks,0,0.0,4793124180,6.847702e+09


In [224]:
wicaractor.nlargest(40,'wicar', keep='first')

,tconst,title,released,nconst,primaryName,boxoffice,inf_adj_boxoffice,cr,wicar
40414,tt3704428,Untitled Elvis Presley Project,06-03-2022,nm0000158,Tom Hanks,0,0.00,4793124180,6.847702e+09
7769,tt10298810,Lightyear,06-17-2022,nm0262635,Chris Evans,0,0.00,4181889128,5.984096e+09
5768,tt10640346,Babylon,01-06-2023,nm0000093,Brad Pitt,0,0.00,2787834786,5.689459e+09
11974,tt8912936,DC League of Super-Pets,05-20-2022,nm0425005,Dwayne Johnson,0,0.00,3875946334,5.537066e+09
11990,tt6443346,Black Adam,07-29-2022,nm0425005,Dwayne Johnson,0,0.00,3875946334,5.537066e+09
40385,tt9603212,Mission: Impossible 7,05-27-2022,nm0000129,Tom Cruise,0,0.00,3856461061,5.509230e+09
16267,tt1462764,Untitled Indiana Jones Project,07-29-2022,nm0000148,Harrison Ford,0,0.00,3569885855,5.101716e+09
30176,tt10648342,Thor: Love and Thunder,05-06-2022,nm0000354,Matt Damon,0,0.00,3102765283,4.432522e+09
6371,tt14174168,Gasoline Alley,02-01-2022,nm0000246,Bruce Willis,0,0.00,2962381840,4.231974e+09
37948,tt6079772,The Protégé,08-20-2021,nm0000168,Samuel L. Jackson,0,0.00,4135546827,4.135547e+09


#### Lets create wicar for directors

In [225]:
filtered_allmerged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 55899 entries, 1453662 to 1285510
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   nconst             55899 non-null  object
 1   primaryName        55899 non-null  object
 2   birthYear          55899 non-null  object
 3   deathYear          55899 non-null  object
 4   primaryProfession  55800 non-null  object
 5   knownForTitles     55899 non-null  object
 6   tconst             55899 non-null  object
 7   ordering           55899 non-null  int64 
 8   category           55899 non-null  object
 9   job                55899 non-null  object
 10  characters         55899 non-null  object
 11  titleType          55899 non-null  object
 12  primaryTitle       55899 non-null  object
 13  originalTitle      55899 non-null  object
 14  isAdult            55899 non-null  object
 15  startYear          55899 non-null  object
 16  endYear            55899 non-nul

In [227]:
filtered_allmerged_df.loc[filtered_allmerged_df.category == 'director']

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles,tconst,ordering,category,job,characters,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
1783002,nm2746751,Aaron Burns,1985,\N,"actor,visual_effects,camera_department","tt3605418,tt1691449,tt1077258,tt2403021",tt1691449,5,director,\N,\N,movie,Blacktino,Blacktino,0,2011,\N,94,Comedy
1783005,nm2746751,Aaron Burns,1985,\N,"actor,visual_effects,camera_department","tt3605418,tt1691449,tt1077258,tt2403021",tt5707304,5,director,\N,\N,movie,Mother,Madre,0,2016,\N,95,"Drama,Horror,Thriller"
76554,nm0004978,Adrian Grenier,1976,\N,"actor,producer,director","tt0458352,tt0387199,tt1674771,tt0164114",tt0323965,5,director,\N,\N,movie,Shot in the Dark,Shot in the Dark,0,2002,\N,88,Documentary
76558,nm0004978,Adrian Grenier,1976,\N,"actor,producer,director","tt0458352,tt0387199,tt1674771,tt0164114",tt1232206,5,director,\N,\N,movie,Teenage Paparazzo,Teenage Paparazzo,0,2010,\N,94,Documentary
2210258,nm7014317,Ahmed Hassan,\N,\N,director,"tt12172364,tt13994410,tt4308488",tt4308488,5,director,\N,\N,movie,Al Makida,Al Makida,0,2014,\N,68,Drama
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225589,nm0103785,Zach Braff,1975,\N,"actor,producer,director","tt0333766,tt0285403,tt2870708,tt1623205",tt14153080,3,director,\N,\N,movie,A Good Person,A Good Person,0,\N,\N,\N,Drama
225594,nm0103785,Zach Braff,1975,\N,"actor,producer,director","tt0333766,tt0285403,tt2870708,tt1623205",tt5294524,2,director,\N,\N,movie,Bump,Bump,0,\N,\N,\N,Drama
191921,nm0077345,Zoe Berriatúa,1978,\N,"actor,writer,director","tt4292596,tt3295030,tt1858803,tt0436225",tt4292596,5,director,\N,\N,movie,Los héroes del mal,Los héroes del mal,0,2015,\N,98,"Crime,Drama,Thriller"
191922,nm0077345,Zoe Berriatúa,1978,\N,"actor,writer,director","tt4292596,tt3295030,tt1858803,tt0436225",tt6984530,5,director,\N,\N,movie,Up Among the Stars,En las estrellas,0,2018,\N,86,Drama


In [234]:
filtered_all_directors_df = filtered_allmerged_df.loc[filtered_allmerged_df.category == 'director']

In [236]:
bigmerge_directors_df = filtered_all_directors_df.merge(queryied_80sto_20sdf,left_on='tconst', right_on='mid')

In [237]:
bigdir_wicar_df = bigmerge_directors_df[['tconst','title','released','nconst','primaryName','boxoffice']]

In [251]:
bigdir_wicar_df.to_excel('./data/bigdir_wicar_df.xls')

In [239]:
filtered_allmerged_df.loc[filtered_allmerged_df.category == 'director']

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles,tconst,ordering,category,job,characters,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
1783002,nm2746751,Aaron Burns,1985,\N,"actor,visual_effects,camera_department","tt3605418,tt1691449,tt1077258,tt2403021",tt1691449,5,director,\N,\N,movie,Blacktino,Blacktino,0,2011,\N,94,Comedy
1783005,nm2746751,Aaron Burns,1985,\N,"actor,visual_effects,camera_department","tt3605418,tt1691449,tt1077258,tt2403021",tt5707304,5,director,\N,\N,movie,Mother,Madre,0,2016,\N,95,"Drama,Horror,Thriller"
76554,nm0004978,Adrian Grenier,1976,\N,"actor,producer,director","tt0458352,tt0387199,tt1674771,tt0164114",tt0323965,5,director,\N,\N,movie,Shot in the Dark,Shot in the Dark,0,2002,\N,88,Documentary
76558,nm0004978,Adrian Grenier,1976,\N,"actor,producer,director","tt0458352,tt0387199,tt1674771,tt0164114",tt1232206,5,director,\N,\N,movie,Teenage Paparazzo,Teenage Paparazzo,0,2010,\N,94,Documentary
2210258,nm7014317,Ahmed Hassan,\N,\N,director,"tt12172364,tt13994410,tt4308488",tt4308488,5,director,\N,\N,movie,Al Makida,Al Makida,0,2014,\N,68,Drama
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225589,nm0103785,Zach Braff,1975,\N,"actor,producer,director","tt0333766,tt0285403,tt2870708,tt1623205",tt14153080,3,director,\N,\N,movie,A Good Person,A Good Person,0,\N,\N,\N,Drama
225594,nm0103785,Zach Braff,1975,\N,"actor,producer,director","tt0333766,tt0285403,tt2870708,tt1623205",tt5294524,2,director,\N,\N,movie,Bump,Bump,0,\N,\N,\N,Drama
191921,nm0077345,Zoe Berriatúa,1978,\N,"actor,writer,director","tt4292596,tt3295030,tt1858803,tt0436225",tt4292596,5,director,\N,\N,movie,Los héroes del mal,Los héroes del mal,0,2015,\N,98,"Crime,Drama,Thriller"
191922,nm0077345,Zoe Berriatúa,1978,\N,"actor,writer,director","tt4292596,tt3295030,tt1858803,tt0436225",tt6984530,5,director,\N,\N,movie,Up Among the Stars,En las estrellas,0,2018,\N,86,Drama


In [250]:
bigdir_wicar_df = queryied_80sto_20sdf[['mid','title','released','director','boxoffice']]